# Buoc 1: Load du lieu

In [5]:
def loaddata(from_date, to_date):
    # Import necessary modules
    from ssi_fc_data import fc_md_client, model
    import pandas as pd  # Import Pandas for DataFrame handling
    import json
    import configDataSSL

    # Create a Market Data Client
    # from_date = "01/11/2023"
    # to_date = "17/11/2023"
    client = fc_md_client.MarketDataClient(configDataSSL)

    req = model.daily_ohlc('VCB', from_date, to_date) # Viet lai lay tu yfinance

    data_dict = client.daily_ohlc(configDataSSL, req)
    print(type(data_dict))
    # Access the list of dictionaries in the "data" field
    data_list = data_dict['data']
    print(data_list)
    # Convert the list of dictionaries into a DataFrame
    data = pd.DataFrame(data_list)

    # Print or work with the DataFrame
    print(data)
    return data

In [10]:
data = loaddata('01/01/2024', '16/01/2024')

<class 'dict'>
[{'Symbol': 'VCB', 'Market': 'HOSE', 'TradingDate': '02/01/2024', 'Time': None, 'Open': '82900', 'High': '83600', 'Low': '82200', 'Close': '83500', 'Volume': '1785800', 'Value': '148183830000'}, {'Symbol': 'VCB', 'Market': 'HOSE', 'TradingDate': '03/01/2024', 'Time': None, 'Open': '83500', 'High': '84500', 'Low': '82800', 'Close': '84500', 'Volume': '1373000', 'Value': '115183170000'}, {'Symbol': 'VCB', 'Market': 'HOSE', 'TradingDate': '04/01/2024', 'Time': None, 'Open': '84500', 'High': '86200', 'Low': '84000', 'Close': '85900', 'Volume': '2657900', 'Value': '227008409999.9990'}, {'Symbol': 'VCB', 'Market': 'HOSE', 'TradingDate': '05/01/2024', 'Time': None, 'Open': '85900', 'High': '86200', 'Low': '85700', 'Close': '86200', 'Volume': '1180300', 'Value': '101478730000'}, {'Symbol': 'VCB', 'Market': 'HOSE', 'TradingDate': '08/01/2024', 'Time': None, 'Open': '86300', 'High': '86800', 'Low': '86300', 'Close': '86800', 'Volume': '1607800', 'Value': '139197059999.9990'}, {'Sy

# Buoc 2: Viet 3 ham kiem tra du lieu

In [11]:
def kiemtratinhieu(data, from_date, to_date):
    import pandas as pd
    # data o tren

    data['TradingDate'] = pd.to_datetime(data['TradingDate'], dayfirst=True)
    data.set_index('TradingDate', inplace=True)
    # Giả sử 'data' là DataFrame của bạn với dữ liệu lịch sử giá cổ phiếu
    data['Open'] = pd.to_numeric(data['Open'], errors='coerce')
    data['High'] = pd.to_numeric(data['High'], errors='coerce')
    data['Low'] = pd.to_numeric(data['Low'], errors='coerce')
    data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
    data['Volume'] = pd.to_numeric(data['Volume'], errors='coerce')

    # Định nghĩa hàm để kiểm tra nến Doji chân dài
    def is_long_legged_doji(row):
        body_range = abs(row['Close'] - row['Open']) # Doji khong phan biet Open > Close hay Close > Open
        upper_shadow = row['High'] - max(row['Open'], row['Close'])
        lower_shadow = min(row['Open'], row['Close']) - row['Low']
        # Điều chỉnh ngưỡng này theo dữ liệu cụ thể của bạn
        doji_threshold = 0.1 / 100 * row['Close']
        return body_range <= doji_threshold and upper_shadow >= 2 * body_range and lower_shadow >= 2 * body_range

    # Định nghĩa hàm để kiểm tra nến tăng
    def is_bullish_candle(current_row, previous_row):
        return (current_row['Close'] > current_row['Open'] and
                current_row['Close'] > previous_row['Close'] and
                previous_row['Close'] <= previous_row['Open'])

    # Định nghĩa hàm để kiểm tra nến giảm
    def is_bearish_candle(current_row, previous_row):
        return (current_row['Close'] < current_row['Open'] and
                current_row['Close'] < previous_row['Close'] and
                previous_row['Close'] >= previous_row['Open'])

    # Lặp qua DataFrame 2 record cuối cùng để kiểm tra chiến lược mua
    buy_signals = []
    sell_signals = []

    for i in range(0, len(data)): # Chi lay 2 nen
        current_row = data.iloc[i]
        previous_row = data.iloc[i - 1]
        
        # Kiểm tra nến hiện tại có phải là nến tăng và nếu nến trước đó là nến Doji chân dài
        if is_bullish_candle(current_row, previous_row) and is_long_legged_doji(previous_row):
            # Nếu thỏa mãn cả ba điều kiện, thêm ngày vào danh sách tín hiệu mua
            buy_signals.append(current_row.name)  # .name sẽ lấy chỉ số (ngày) của hàng
        if is_bearish_candle(current_row, previous_row) and is_long_legged_doji(previous_row):
            sell_signals.append(current_row.name)

    # In ra danh sách các tín hiệu mua
    print("Buy signals " + from_date + " to " + to_date + "on dates:", buy_signals)
    print("Sell signals on dates:", sell_signals)

# Buoc 3: Show nen len de xem

In [12]:
def showdata(data):
    import matplotlib.pyplot as plt
    import matplotlib.lines as mlines

    def draw_candlestick_with_labels(ax, data):
        # Sử dụng enumerate để nhận vị trí chỉ số và dữ liệu hàng
        for i, (index, row) in enumerate(data.iterrows()):
            # Xác định màu sắc dựa trên giá Open và Close
            color = 'green' if row['Open'] <= row['Close'] else 'red'
            
            # Vẽ bóng nến (High và Low)
            ax.plot([i, i], [row['Low'], row['High']], color='black')
            
            # Vẽ thân nến (Open và Close)
            body_height = max(row['Close'] - row['Open'], 0.01)  # Đảm bảo có chiều cao tối thiểu
            body_bottom = min(row['Close'], row['Open'])
            open_close_rect = plt.Rectangle((i - 0.2, body_bottom), 0.4, body_height, 
                                            facecolor=color, edgecolor='black')
            ax.add_patch(open_close_rect)
            
            # Thêm nhãn giá
            ax.text(i, row['High'], f"{row['High']}", va='bottom', ha='center', fontsize=8)
            ax.text(i, row['Low'], f"{row['Low']}", va='top', ha='center', fontsize=8)
            ax.text(i - 0.2, row['Open'], f"{row['Open']}", va='center', ha='right', fontsize=8)
            ax.text(i + 0.2, row['Close'], f"{row['Close']}", va='center', ha='left', fontsize=8)

    # Khởi tạo plot
    fig, ax = plt.subplots(figsize=(10, 6))

    # Vẽ các nến với nhãn từ DataFrame
    draw_candlestick_with_labels(ax, data)

    # Thiết lập giới hạn cho biểu đồ
    ax.set_xlim(-1, len(data))
    ax.set_ylim(data['Low'].min() * 0.95, data['High'].max() * 1.05)  # Đặt giới hạn với một chút đệm

    # Thiết lập nhãn cho trục x
    ax.set_xticks(range(len(data)))
    ax.set_xticklabels([index.strftime('%Y-%m-%d') for index in data.index], rotation=45)
    ax.set_ylabel('Price')

    # Hiển thị plot
    plt.show()


# Auto Trade

In [14]:
import schedule
import time
from datetime import datetime, timedelta

def execute_trade():
    end_date = datetime.now() # Lấy ngày hiện tại
    start_date = end_date - timedelta(days=28)  # Lấy ngày trước ngày hiện tại

    # Định dạng ngày tháng theo chuẩn (ví dụ: "16-11-2023")
    start_date_str = start_date.strftime("%d/%m/%Y")
    end_date_str = end_date.strftime("%d/%m/%Y")

    data = loaddata(start_date_str, end_date_str)
    # print(data)
    kiemtratinhieu(data, start_date_str, end_date_str)

# Lên lịch hàm execute_trade để chạy mỗi phút
# schedule.every(1).minute.do(execute_trade)
schedule.every(1).day.at("20:53").do(execute_trade)

while True:
    schedule.run_pending()  # Kiểm tra và thực hiện các công việc lên lịch
    time.sleep(1)

<class 'dict'>
[{'Symbol': 'VCB', 'Market': 'HOSE', 'TradingDate': '19/12/2023', 'Time': None, 'Open': '81500', 'High': '81500', 'Low': '80100', 'Close': '81200', 'Volume': '1961300', 'Value': '158554710000'}, {'Symbol': 'VCB', 'Market': 'HOSE', 'TradingDate': '20/12/2023', 'Time': None, 'Open': '81200', 'High': '81300', 'Low': '80500', 'Close': '81100', 'Volume': '1872300', 'Value': '151412890000.0010'}, {'Symbol': 'VCB', 'Market': 'HOSE', 'TradingDate': '21/12/2023', 'Time': None, 'Open': '81000', 'High': '81000', 'Low': '80300', 'Close': '80900', 'Volume': '2030300', 'Value': '163668459999.9990'}, {'Symbol': 'VCB', 'Market': 'HOSE', 'TradingDate': '22/12/2023', 'Time': None, 'Open': '80900', 'High': '81100', 'Low': '80400', 'Close': '80900', 'Volume': '1579800', 'Value': '127435990000.0010'}, {'Symbol': 'VCB', 'Market': 'HOSE', 'TradingDate': '25/12/2023', 'Time': None, 'Open': '80900', 'High': '81800', 'Low': '80700', 'Close': '81800', 'Volume': '1301600', 'Value': '106015319999.99